# A hidden Markov model for lymphatic tumour progression in head and neck cancer

Roman Ludwig¹*, Bertrand Pouymayou¹, Panagiotis Balermpas¹ and Jan Unkelbach¹

¹ Departement of Radiation Oncology, University Hospital Zurich, Switzerland \
\* [roman.ludwig@usz.ch](mailto:roman.ludwig@usz.ch)

***

## Abstract

Currently , elective clinical target volume (CTV-N) definition for head & neck squamous cell carcinoma (HNSCC) is mostly based on the prevalence of nodal involvement for a given tumor location. In this work, we propose a probabilistic model for lymphatic metastatic spread that can quantify the risk of microscopic involvement in lymph node levels (LNL) given the location of macroscopic metastases and T-stage. This may allow for further personalized CTV-N definition based on an individual patient’s state of disease. \
We model the patient's state of metastatic lymphatic progression as a collection of hidden binary random variables that indicate the involvement of LNLs. In addition, each LNL is associated with observed binary random variables that indicate whether macroscopic metastases are detected. A hidden Markov model (HMM) is used to compute the probabilities of transitions between states over time. The underlying graph of the HMM represents the anatomy of the lymphatic drainage system. Learning of the transition probabilities is done via Markov chain Monte Carlo sampling and is based on a dataset of HNSCC patients in whom involvement of individual LNLs was report-ed. \
The model is demonstrated for ipsilateral metastatic spread in oropharyngeal HNSCC patients. We demonstrate the model's capability to quantify the risk of microscopic involvement in levels III and IV, depending on whether macroscopic metastases are observed in the upstream levels II and III, and depending on T-stage. \
In conclusion, the statistical model of lymphatic progression may inform future, more personal-ized, guidelines on which LNL to include in the elective CTV. However, larger multi-institutional datasets for model parameter learning are required for that. 

***

## Introduction

This notebook contains all the code we ran to produce results and plots for our paper. It is intended to be read alongside the paper if one wants to understand better how exactly we implemented and used the methodology introduced in the paper. However, this notebook is NOT a stand-alone work but only a supplement.

### Imports

First, we import some libraries that are necessary for our implementation. [`lymph`](https://github.com/rmnldwg/lymph) is the package we wrote, while [`corner`](https://corner.readthedocs.io/en/latest/) and [`emcee`](https://emcee.readthedocs.io/en/stable/) (see also the corresponding [arXiv paper](https://arxiv.org/abs/1202.3665) on this package) are both packages by Dan Foreman-Mackey & contributors. [`matplotlib`](https://matplotlib.org/stable/index.html) is an extensive and powerful plotting library. All other packages are standard and included with any default python installation.

In [ ]:
# basic stuff
import numpy as np
import scipy as sp
from scipy import stats
import pandas as pd
from multiprocessing import Pool
import datetime as dt

# plotting
import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
import matplotlib.font_manager as font_manager
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import ListedColormap
from cycler import cycler
import corner

# sampling
import emcee
import tqdm

# our package
import lymph

## Settings

The variables below are meant to be constants. `T_MAX` is the length of the used binomial time-prior. `DRAW_SAMPLES` is a `bool` that defines whether new samples will be drawn for the plots and computations or alreadz drawn samples should be loaded from file. `SEED` is the seed for the random number generator in `numpy` and aims at reproducability. `SAVE_FIGURES` defines whether or not generated figures should be saved to disk.

In [ ]:
T_MAX = 10
DRAW_SAMPLES = True
SEED = 42
SAVE_FIGURES = True

### Colors

We chose the colors of the University Hospital Zurich's corporate design for the default colors of our plots.

In [ ]:
# USZ colors
usz_blue = '#005ea8'
usz_green = '#00afa5'
usz_red = '#ae0060'
usz_orange = '#f17900'
usz_gray = '#c5d5db'

# colormaps
white_to_blue  = LinearSegmentedColormap.from_list("white_to_blue", 
                                                   ["#ffffff", usz_blue], 
                                                   N=256)
white_to_green = LinearSegmentedColormap.from_list("white_to_green", 
                                                   ["#ffffff", usz_green], 
                                                   N=256)
green_to_red   = LinearSegmentedColormap.from_list("green_to_red", 
                                                   [usz_green, usz_red], 
                                                   N=256)

h = usz_gray.lstrip('#')
gray_rgba = tuple(int(h[i:i+2], 16) / 255. for i in (0, 2, 4)) + (1.0,)
tmp = LinearSegmentedColormap.from_list("tmp", [usz_green, usz_red], N=128)
tmp = tmp(np.linspace(0., 1., 128))
tmp = np.vstack([np.array([gray_rgba]*128), tmp])
halfGray_halfGreenToRed = ListedColormap(tmp)

### Plot Settings

Here we define a function to consistently set the size of our plots and we load the default settings regarding font size etc from an `.mplstyle` file.

In [ ]:
plt.style.use(".mplstyle")

def set_size(width="single", unit="cm", ratio="golden"):
    if width == "single":
        width = 10
    elif width == "full":
        width = 16
    else:
        try:
            width = width
        except:
            width = 10
            
    if unit == "cm":
        width = width / 2.54
        
    if ratio == "golden":
        ratio = 1.618
    else:
        ratio = ratio
    
    try:
        height = width / ratio
    except:
        height = width / 1.618
        
    return (width, height)

## Data

The dataset we use here was reconstructed by [Pouymayou et al.](#pouymayou) from [Sanguineti et al.](#sanguineti). Here, it is loaded from file. Note that $N_0$ patients were added to make up 30% of the dataset.

In [ ]:
# parameters and risk estimation from Pouymayou et al [2]
pouymayou_params = [0.061, 0.638, 0.094, 0.057, 0.08, 0.331, 0.242]
pouymayou_MLrisk = np.array([[ 1.53,  1.64,  1.66,  1.56], 
                             [24.67, 81.55, 89.64, 39.06], 
                             [ 4.48,  9.97, 59.93, 38.75], 
                             [ 1.83,  2.25,  6.05,  4.44]]) / 100.

# data reconstructed from Sanguineti et al [1] (without N0 patients)
data = pd.read_csv("./data/2009_sanguineti.csv", 
                   header=None, 
                   names=['I', 'II', 'III', 'IV'])

# inserting info about the "T-stage"
data.insert(0, "T-stage", ["early"] * data.shape[0])

columns = pd.MultiIndex.from_arrays([['Info', 'path', 'path', 'path', 'path'], 
                                     ['T-stage', 'I', 'II', 'III', 'IV']])
data = pd.DataFrame(data.values.tolist(), columns=columns)
data

## Inference

In this section we will set up everything necessary to perform inference on the dataset recostructed by [[2]](#pouymayou) from [[1]](#sanguineti).

### Lymphatic Network

Here, we need to define the underlying anatomical network of lymph node levels (LNLs), as it is also defined in [[2]](#pouymayou). 

The tumor and every LNL are represented by a key in a dictionary called `graph` each. The respective value in the dictionary is a list of nodes it drains to.

In [ ]:
# abstract representation of the lymphatic network
graph = {'T'  : ['I', 'II', 'III', 'IV'], 
         'I'  : ['II'], 
         'II' : ['III'], 
         'III': ['IV'], 
         'IV' : []}

systm = lymph.System(graph=graph)

### Time Prior
We need to choose a time prior for the parameter learning. A [Binomial distribution](https://en.wikipedia.org/wiki/Binomial_distribution) was chosen for its intuitively meaningful shape and simple structure.

In [ ]:
fig, ax = plt.subplots(figsize=set_size());

for i,p in enumerate([0.4, 0.55, 0.7]):
    ax.plot(np.arange(T_MAX+1), 
            sp.stats.binom.pmf(np.arange(T_MAX+1), T_MAX, p), 
            "o-", label=f"$p = {{{p}}}$")
    
ax.set_xlim([0,10]);
ax.set_ylim([-0.02,0.35])
ax.set_xlabel("time step $t$");
ax.set_ylabel(r"$p(t)$");
ax.tick_params();
ax.legend();

In [ ]:
# creating a dictionary of time priors for the sampling process
time_prior_dict = {}
time_prior_dict["early"] = sp.stats.binom.pmf(np.arange(T_MAX+1), T_MAX, 0.4)

### Learning (HMM)

Using the anatomical model along with the data, we can now load the data into the lymph system class. To do so we must also pass a dictionary called `spsn_dict`. **``spsn``** stands for **sp**ecificity and **s**e**n**sitivity. In that dictionary, one has to define a list ``[specificty, sensitivity]`` for each diagnostic modality one is interested. So if we, for example, have MRI and CT data, then we would pass a dictionary like this:

```python
spsn_dict = {"MRI": [spec_MRI, sens_MRI], 
             "CT" : [spec_CT , sens_CT ]}
```

Note however, that the keys of this dictionary must be diagnostic modalities that are also present in the dataset. More precisely, they must be the overarching categories in the ``MultiIndex`` under which one then finds the individual LNLs.

Finally, we can use the likelihood function that is built into the `lymph` package together with the sampling implementation `emcee` to infer the base probabilities $b_{v}$ and transition probabilities $t_{\text{pa}(v)v}$.

In [ ]:
# define specificity and sensitivity for diagnostic modalities
spsn_dict = {"path": [1., 1.]}

# load data
systm.load_data(data, t_stage=["early"], spsn_dict=spsn_dict, mode="HMM")

# check if likelihood works
assert systm.likelihood(np.random.uniform(size=(7,)), 
                        t_stage=["early"], 
                        time_prior_dict=time_prior_dict, 
                        mode="HMM")

In [ ]:
# parameters of the sampler
ndim, nwalker, nstep, burnin = 7, 200, 2000, 1000
moves = [(emcee.moves.DEMove(), 0.8), (emcee.moves.DESnookerMove(), 0.2)]

if DRAW_SAMPLES:
    # starting point
    np.random.seed(SEED)
    theta0 = np.random.uniform(low=0., high=1., size=(nwalker,ndim))

    # the actual sampling round
    if __name__ == "__main__":
        with Pool() as pool:
            sampler = emcee.EnsembleSampler(nwalker, ndim, systm.likelihood, 
                                            args=[['early'], time_prior_dict], 
                                            moves=moves, pool=pool)
            sampler.run_mcmc(theta0, nstep, progress=True)

    # extracting 200,000 of the 400,000 samples
    samples_HMM = sampler.get_chain(flat=True, discard=burnin)

    # saving the sampled data to disk for later convenience
    np.save("./samples/HMM.npy", samples_HMM)
    
else:
    # loading in case we don't want to draw all the samples again
    samples_HMM = np.load("./samples/HMM.npy")

In [ ]:
if DRAW_SAMPLES:
    # check acceptance faction of the sampler to get an indication on whether sth
    # went wrong or not
    ar = np.mean(sampler.acceptance_fraction)
    print(f"the HMM sampler accepted {ar * 100 :.2f} % of samples.")

In [ ]:
labels = [r"$\tilde{b}_1$", r"$\tilde{b}_2$", 
          r"$\tilde{b}_3$", r"$\tilde{b}_4$", 
          r"$\tilde{t}_{12}$", r"$\tilde{t}_{23}$", r"$\tilde{t}_{34}$"]

fig = plt.figure(figsize=set_size(width="full", ratio=1));

# using the corner plot package
corner.corner(
    samples_HMM,
    labels=labels,
    smooth=True,
    fig=fig,
    color=usz_blue,
    hist_kwargs={'histtype': 'stepfilled', 'color': usz_blue},
    **{
        "plot_datapoints": False,
        "plot_density": True,
        "no_fill_contours": True,
        "contour_kwargs": {"colors": "black"},
        "levels": np.array([0.2, 0.5, 0.8])
    },
    show_titles=True,
    title_kwargs={"fontsize": "medium"}
);

axes = fig.get_axes();
for ax in axes:
    ax.grid(False);
    
if SAVE_FIGURES:
    plt.savefig("./figures/corner_HMM.png", dpi=300, bbox_inches="tight");
    plt.savefig("./figures/corner_HMM.svg", bbox_inches="tight");

### Transition Matrix

We can now set the hidden Markov model's parameters to the expected value of the inferred parameters and look at the resulting transition matrix $\mathbf{A}$.

In [ ]:
# set the parameters
systm.set_theta(np.mean(samples_HMM, axis=0))

# modify the transition matrix for nicer coloring
mod_A = -1 * np.ones_like(systm.A)
for key, nums in systm._idx_dict.items():
    for i in nums:
        mod_A[key, i] = systm.A[key, i]

# plot the transition matrix
fig, ax = plt.subplots(figsize=set_size(ratio=1.));

h = ax.imshow(mod_A, cmap=halfGray_halfGreenToRed, vmin=-1., vmax=1.);
ax.set_xticks(range(len(systm.state_list)));
ax.set_xticklabels(systm.state_list, rotation=-90, fontsize="small");
ax.set_yticks(range(len(systm.state_list)));
ax.set_yticklabels(systm.state_list, fontsize="small");
ax.tick_params(direction="out")
ax.grid(False)

# label the non-zero entries with their probability in %
for i in range(len(systm.state_list)):
    for j in range(len(systm.state_list)):
        if mod_A[i,j] > 0.:
            ax.text(j,i, f"{mod_A[i,j]*100:.1f}", ha="center", va="center", 
                    color="white", fontsize="x-small")
            
if SAVE_FIGURES:
    plt.savefig("./figures/transition_matrix.png", dpi=300, bbox_inches="tight")
    plt.savefig("./figures/transition_matrix.svg", bbox_inches="tight")

### Evolution Plots

We can also take a look at how this system evolves over the defined time steps.

In [ ]:
# array containing the risk for each state...
state_array = np.zeros(shape=(T_MAX+1, len(systm.state_list)), dtype=float)
# ...weighted with the probability for that time
state_array_weighted = np.zeros_like(state_array, dtype=float)
state_array_summed = np.zeros_like(state_array, dtype=float)
# starting state
start = np.zeros(shape=(len(systm.state_list),))
start[0] = 1.

# manually evolving the system and storing all intermediate states
time_prior = sp.stats.binom.pmf(np.arange(T_MAX+1), T_MAX, 0.4)
for t,p in enumerate(time_prior):
    state_array[t] = start
    state_array_weighted[t] = p * start
    state_array_summed[t] = np.ones(shape=(1,t+1)) @ state_array_weighted[:t+1]
    state_array_summed[t] = state_array_summed[t] / np.sum(state_array_summed[t])
    start = start @ systm.A

In [ ]:
# these arrays define which states we need to marginalize over when we are 
# interested in a particual LNL's risk of involvement
lnl_I_arr = np.array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 
                     dtype=float)
lnl_II_arr = np.array([0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1], 
                      dtype=float)
lnl_III_arr = np.array([0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1], 
                       dtype=float)
lnl_IV_arr = np.array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1], 
                      dtype=float)

lnl_I = state_array @ lnl_I_arr
lnl_II = state_array @ lnl_II_arr 
lnl_III = state_array @ lnl_III_arr 
lnl_IV = state_array @ lnl_IV_arr

lnl_concat = np.vstack([lnl_I, lnl_II, lnl_III, lnl_IV])

In [ ]:
marg_II = np.zeros(shape=(len(samples_HMM[::50])))
marg_III = np.zeros(shape=(len(samples_HMM[::50])))
marg_II_and_III = np.zeros(shape=(len(samples_HMM[::50])))
marg_IV = np.zeros(shape=(len(samples_HMM[::50])))

np.random.seed(SEED)
for i, theta in enumerate(np.random.permutation(samples_HMM[::50])):
    systm.set_theta(theta)
    marg_II[i] = systm.risk(inv=np.array([None, 1, None, None]),
                            obs={"path": np.array([None, None, None, None])},
                            time_prior=time_prior) * 100
    marg_III[i] = systm.risk(inv=np.array([None, None, 1, None]),
                             obs={"path": np.array([None, None, None, None])},
                             time_prior=time_prior) * 100
    marg_II_and_III[i] = systm.risk(inv=np.array([None, 1, 1, None]),
                                    obs={"path": np.array([None, None, None, None])},
                                    time_prior=time_prior) * 100
    marg_IV[i] = systm.risk(inv=np.array([None, None, None, 1]),
                            obs={"path": np.array([None, None, None, None])},
                            time_prior=time_prior) * 100

In [ ]:
# marginalization for involvements of "at least LNL x" or "only LNL x" 
only_II = state_array[:,4]
atleast_II = (state_array[:,4] + state_array[:,5] + state_array[:,6] 
              + state_array[:,7] + state_array[:,12] + state_array[:,13] 
              + state_array[:,14] + state_array[:,15])
emp_II = 100 * np.sum(data[("path", 'II')].to_numpy()) / 147

only_III = state_array[:,2]
atleast_III = (state_array[:,2] + state_array[:,3] + state_array[:,6] 
               + state_array[:,7] + state_array[:,10] + state_array[:,11] 
               + state_array[:,14] + state_array[:,15])
emp_III = 100 * np.sum(data[("path", 'III')].to_numpy()) / 147

only_II_and_III = state_array[:,6]
atleast_II_and_III = (state_array[:,6] + state_array[:,7] 
                      + state_array[:,14] + state_array[:,15])
emp_II_and_III = 100 * len(data.loc[(data[("path", 'II')]==1) 
                                    & (data[("path", 'III')]==1)].to_numpy()) / 147

only_IV = state_array[:,1]
atleast_IV = (state_array[:,1] + state_array[:,3] + state_array[:,5] 
              + state_array[:,7] + state_array[:,9] + state_array[:,11] 
              + state_array[:,13] + state_array[:,15])
emp_IV = 100 * np.sum(data[("path", 'IV')].to_numpy()) / 147

# and now for one complicated plot...
fig = plt.figure(figsize=set_size(width="full", ratio=2*1.61));
spec = gs.GridSpec(ncols=3, nrows=1, figure=fig, width_ratios=[1., 1., 0.3]);

# leftmost subplot
ax = fig.add_subplot(spec[0,0])
ax.plot(range(len(time_prior)), 100*only_II, 'o-', 
        label=r"$\xi_5=[0\ 1\ 0\ 0]$");
ax.plot(range(len(time_prior)), 100*only_III, 'o-', 
        label=r"$\xi_3=[0\ 0\ 1\ 0]$");
ax.plot(range(len(time_prior)), 100*only_II_and_III, 'o-', 
        label=r"$\xi_7=[0\ 1\ 1\ 0]$");
ax.plot(range(len(time_prior)), 100*only_IV, 'o-', 
        label=r"$\xi_2=[0\ 0\ 0\ 1]$");
ax.set_xlabel("time step $t$");
ax.set_ylim(ymax=50);
ax.set_ylabel("Risk [%]");
ax.legend();

# middle subplot
ax = fig.add_subplot(spec[0,1])
ax.plot(range(len(time_prior)), 100*atleast_II, 'o-', 
        label="lvl II involved");
ax.plot(range(len(time_prior)), 100*atleast_III, 'o-', 
        label="lvl III involved");
ax.plot(range(len(time_prior)), 100*atleast_II_and_III, 'o-', 
        label="lvl II & III involved");
ax.plot(range(len(time_prior)), 100*atleast_IV, 'o-', 
        label="lvl IV involved");
ax.set_xlabel("time step $t$");
ax.set_ylim(ymax=100);
ax.legend();

# rightmost subplot
ax = fig.add_subplot(spec[0,2], sharey=ax);
plt.setp(ax.get_yticklabels(), visible=False);
ax.set_xticks([0, 1, 2, 3]);
ax.set_xticklabels(["II", "III", "II & III", "IV"], rotation=-45);

violin = ax.violinplot(marg_II, positions=[0]);
violin["bodies"][0].set_color(usz_blue);
violin["cbars"].set_color(usz_blue);
ax.axhline(emp_II, color=usz_blue, ls="--");

violin = ax.violinplot(marg_III, positions=[1]);
violin["bodies"][0].set_color(usz_orange);
violin["cbars"].set_color(usz_orange);
ax.axhline(emp_III, color=usz_orange, ls="--");

violin = ax.violinplot(marg_II_and_III, positions=[2]);
violin["bodies"][0].set_color(usz_red);
violin["cbars"].set_color(usz_red);
ax.axhline(emp_II_and_III, color=usz_red, ls="--");

violin = ax.violinplot(marg_IV, positions=[3]);
violin["bodies"][0].set_color(usz_green);
violin["cbars"].set_color(usz_green);
ax.axhline(emp_IV, color=usz_green, ls="--");

if SAVE_FIGURES:
    plt.savefig("./figures/HMM_evolution.png", dpi=300, bbox_inches="tight")
    plt.savefig("./figures/HMM_evolution.svg", bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(
    1,2,
    sharey=True,
    figsize=set_size(width="full", ratio=1.8),
    constrained_layout=False,
    gridspec_kw={
        "width_ratios": [1., 0.35],
        "wspace": 0.05,
        "height_ratios": [1.]
    }
)

ax[0].set_title("Probabilities [%] of states at different time steps")
ax[0].imshow(state_array, cmap=green_to_red);
for i in range(len(time_prior)):
    for j in range(len(systm.state_list)):
        if np.around(state_array[i,j]*100,1) >= 1.:
            ax[0].text(j,i, f"{state_array[i,j]*100:.1f}", 
                       ha="center", va="center", 
                       color="white", fontsize="xx-small")
ax[0].set_xticks(range(len(systm.state_list)))
ax[0].set_xticklabels(systm.state_list, rotation=45);
ax[0].set_ylabel("time step $t$")
ax[0].set_xlabel(r"state $\xi$")
ax[0].grid(False)
ax0_pos = ax[0].get_position()

ax[1].set_title("Time prior (PDF)")
ax[1].plot(time_prior, range(len(time_prior)), "o-")
ax1_pos = ax[1].get_position()
ax[1].set_position(
    [ax1_pos.x0,
     ax0_pos.y0,
     ax1_pos.x1 - ax1_pos.x0,
     ax0_pos.y1 - ax0_pos.y0]
)
plt.setp(ax[1].get_yticklabels(), visible=False);

if SAVE_FIGURES:
    plt.savefig("./figures/HMM_evo_matrix.png", dpi=300, bbox_inches="tight")
    plt.savefig("./figures/HMM_evo_matrix.svg", bbox_inches="tight")

## (Cross-)Validation

As an attempt to validate the model with the limited data we have, I'll start by simply comparing the prevalence of certain patterns of involvement to the prediction of the model for the respective state.

In [ ]:
thin = 100
np.random.seed(SEED)

risks = np.zeros(shape=(len(systm.obs_list), len(samples_HMM[::thin])), dtype=float)

for i, sample in enumerate(np.random.permutation(samples_HMM[::thin])):
    for j, obs in enumerate(systm.obs_list):
        systm.set_theta(sample)
        risks[j,i] = systm.risk(inv=obs, obs={"path": [None, None, None, None]}, 
                                time_prior=time_prior_dict["early"], mode="HMM")

In [ ]:
occurences = np.zeros(shape=(len(systm.obs_list),), dtype=int)
for i,obs in enumerate(systm.obs_list):
    if np.sum(systm.C_dict["early"][i]) == 0:
        occurences[i] = 0
    else:
        idx = np.argmax(systm.C_dict["early"][i])
        occurences[i] = systm.f_dict["early"][idx]

validation_df = pd.DataFrame({"state": [str(obs) for obs in systm.obs_list], 
                              "occurence": occurences, 
                              "percentage": 100 * occurences / np.sum(occurences), 
                              "prediction": 100 * np.mean(risks, axis=1)})
validation_df

The next thing I could do is to randomly split the dataset into two parts and train the model on the first half, then generating the above table for the second half. Then do it the other way around and/or split the whole dataset randomly again. What's now left to find is a measure of the difference *prevalence* vs *predicted risk*.

## Comparison to Bayesian Network

To be able to compare our results to the Bayesian network by [Pouymayou et al.](#pouymayou) we needed to recreate it using the same sampler. To this end, the `lymph` package also supports computing the Bayesian network likelihood for a given graph and observational modality.

In [ ]:
# abstract representation of the lymphatic network
graph = {'T'  : ['I', 'II', 'III', 'IV'], 
         'I'  : ['II'], 
         'II' : ['III'], 
         'III': ['IV'], 
         'IV' : []}

systm = lymph.System(graph=graph)

### Learning (BN)

All that is different to the learning round [above](#Learning-(HMM)) is that one has to specify the `mode` to be `"BN"` instead of `"HMM"`.

In [ ]:
# define specificity and sensitivity for diagnostic modalities
spsn_dict = {"path": [1., 1.]}

# generate C matrix from data
systm.load_data(data, spsn_dict=spsn_dict, mode="BN")

# check if likelihood works
assert systm.likelihood(np.random.uniform(size=(7,)), mode="BN")

In [ ]:
# parameters of the sampler
ndim, nwalker, nstep, burnin = 7, 200, 2000, 1000
moves = [(emcee.moves.DEMove(), 0.8), (emcee.moves.DESnookerMove(), 0.2)]

if DRAW_SAMPLES:
    # starting point
    np.random.seed(SEED)
    theta0 = np.random.uniform(low=0., high=1., size=(nwalker,ndim))

    # the actual sampling round
    if __name__ == "__main__":
        with Pool() as pool:
            sampler = emcee.EnsembleSampler(nwalker, ndim, systm.likelihood, 
                                            kwargs={"mode": "BN"},
                                            moves=moves, pool=pool)
            sampler.run_mcmc(theta0, nstep, progress=True)

    # extracting 200,000 of the 400,000 samples
    samples_BN = sampler.get_chain(flat=True, discard=burnin)

    # saving the sampled data to disk for later convenience
    np.save("./samples/BN.npy", samples_BN)
    
else:
    # loading in case we don't want to draw all the samples again
    samples_BN = np.load("./samples/BN.npy")

In [ ]:
if DRAW_SAMPLES:
    # check acceptance faction of the sampler to get an indication on whether sth
    # went wrong or not
    ar = np.mean(sampler.acceptance_fraction)
    print(f"the BN sampler accepted {ar * 100 :.2f} % of samples.")

In [ ]:
labels = [r"$b_1$", r"$b_2$", r"$b_3$", r"$b_4$", 
          r"$t_{12}$", r"$t_{23}$", r"$t_{34}$"]

fig = plt.figure(figsize=set_size(width="full", ratio=1))

# using the corner plot package
corner.corner(
    samples_BN,
    labels=labels,
    smooth=True,
    fig=fig,
    color=usz_green,
    hist_kwargs={'histtype': 'stepfilled', 'color': usz_green},
    **{
        "plot_datapoints": False,
        "plot_density": True,
        "no_fill_contours": True,
        "contour_kwargs": {"colors": "black"},
        "levels": np.array([0.2, 0.5, 0.8])
    },
    show_titles=True,
    title_kwargs={"fontsize": "medium"}
);

axes = fig.get_axes()
for ax in axes:
    ax.grid(False)
    
if SAVE_FIGURES:
    plt.savefig("./figures/corner_BN.png", dpi=300, bbox_inches="tight")
    plt.savefig("./figures/corner_BN.svg", bbox_inches="tight")

## Risk Predictions

We can now compute distributions over risks using both the HMM model, as well as the BN.

### Evolving beyond "early" T-stage

We can use the parameters inferred from the early T-stage dataset and use time priors that expect to see a patient's diagnose later on to estimate how risks of involvement might increase over time.

In [ ]:
# abstract representation of the lymphatic network
graph = {'T'  : ['I', 'II', 'III', 'IV'], 
         'I'  : ['II'], 
         'II' : ['III'], 
         'III': ['IV'], 
         'IV' : []}

tst_systm = lymph.System(graph=graph)

# set specificity & sensitivity of diagnostic modality (here CT) manually
spsn_dict = {"CT": [0.76, 0.81]}
tst_systm.set_modalities(spsn_dict=spsn_dict)

In [ ]:
# time priors
time_prior_dict = {}
time_prior_dict['early'] = sp.stats.binom.pmf(np.arange(T_MAX+1), T_MAX, 0.4)
time_prior_dict['mid'] = sp.stats.binom.pmf(np.arange(T_MAX+1), T_MAX, 0.55)
time_prior_dict['late'] = sp.stats.binom.pmf(np.arange(T_MAX+1), T_MAX, 0.7)

# what do we want to know, what do we know?
inv = np.array([None, None, 1, None])  # we're interested in the risk of LNL 3
# our observation is that lvl 2 is involved
obs = {"CT": np.array([0, 1, 0, 0])}

thin = 50
# risk for HMM and different "T-stages"
early = []
mid = []
late = []
np.random.seed(SEED)
for sample in np.random.permutation(samples_HMM)[::thin]:
    tst_systm.set_theta(sample)
    early.append(tst_systm.risk(inv=inv, obs=obs, 
                                time_prior=time_prior_dict["early"], 
                                mode="HMM"))
    mid.append(tst_systm.risk(inv=inv, obs=obs, 
                              time_prior=time_prior_dict["mid"], 
                              mode="HMM"))
    late.append(tst_systm.risk(inv=inv, obs=obs, 
                               time_prior=time_prior_dict["late"], 
                               mode="HMM"))

# risk for BN
bn = []
np.random.seed(SEED)
for sample in np.random.permutation(samples_BN)[::thin]:
    tst_systm.set_theta(sample)
    bn.append(tst_systm.risk(inv=inv, obs=obs, mode="BN"))

In [ ]:
bins = np.linspace(0, 35, 50)
r = (0, 35)
fig, ax = plt.subplots(figsize=set_size())

ax.hist(np.asarray(early)*100., bins=bins, density=True, 
        histtype="stepfilled", color=usz_green, label="$p = 0.4$");
ax.hist(np.asarray(mid)*100., bins=bins, density=True, alpha=0.8, 
        histtype="stepfilled", color=usz_orange, label="$p = 0.55$");
ax.hist(np.asarray(late)*100., bins=bins, density=True, alpha=0.8, 
        histtype="stepfilled", color=usz_red, label="$p = 0.7$");
ax.hist(np.asarray(bn)*100., bins=bins, histtype="step", density=True, 
        color=usz_blue, label="Bayesian network");

ax.set_xlim(r)
ax.set_xlabel("risk $R$ [%]");
ax.set_ylabel(r"$p(R)$");
ax.tick_params();
ax.legend();

if SAVE_FIGURES:
    plt.savefig("./figures/HMM_risk_increaseP.png", dpi=300, 
                bbox_inches="tight")
    plt.savefig("./figures/HMM_risk_increaseP.svg", 
                bbox_inches="tight")

### Comparison to [Pouymayou et al.](#pouymayou)

Now we will compare how the sampled HMM's, sampled BN's and maximum likelihood BN's risk predictions compare to each other.

In [ ]:
ndim, nwalker, nstep, burnin = 7, 200, 2000, 1000
thin = 50

# what do we want to know?
inv = np.array([[1   , None, None, None],
                [None, 1   , None, None],
                [None, None, 1   , None],
                [None, None, None, 1   ]])

# what do we know?
obs = np.array([[0, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 1, 1, 0],
                [0, 0, 1, 0]])

# risk for HMM and two different "T-stages" (early and late)
np.random.seed(SEED)
hmm_risk = np.zeros(shape=(4,4,(nstep-burnin)*nwalker//thin))
for i, sample in enumerate(np.random.permutation(samples_HMM)[::thin]):
    tst_systm.set_theta(sample)
    for k in range(4):
        for l in range(4):
            hmm_risk[k,l,i] = tst_systm.risk(inv=inv[k], obs={"CT": obs[l]}, 
                                             time_prior=time_prior_dict["early"], 
                                             mode="HMM")

# risk for BN
np.random.seed(SEED)
bn_risk = np.zeros(shape=(4,4,(nstep-burnin)*nwalker//thin))
for i, sample in enumerate(np.random.permutation(samples_BN)[::thin]):
    tst_systm.set_theta(sample)
    for k in range(4):
        for l in range(4):
            bn_risk[k,l,i] = tst_systm.risk(inv=inv[k], obs={"CT": obs[l]}, 
                                            mode="BN")

In [ ]:
fig = plt.figure(figsize=set_size(width="full"), constrained_layout=False)
spec = gs.GridSpec(ncols=4, nrows=4, figure=fig, wspace=0.1)

lvls = ["I", "II", "III", "IV"]
txt = ["Ø", "II", "II & III", "III"]
risk_map = LinearSegmentedColormap.from_list("risk_map", [usz_green, 
                                                          usz_gray, 
                                                          usz_red, 
                                                          usz_red], N=256)
n_bins = 25

for i in range(4):
    if (i == 0) or (i == 3):
        bins = np.linspace(0., 12., 30)
        for j in range(4):
            if j == 0:
                ax = fig.add_subplot(spec[i,j])
                ax.set_ylabel(f"{lvls[i]}");
            else:
                ax = fig.add_subplot(spec[i,j], sharey=ax)
                plt.setp(ax.get_yticklabels(), visible=False)
                
            ax.set_xlim(bins[0], bins[-1])
            
            tmp_mean = np.mean(hmm_risk[i,j])
            hmm_color = risk_map(tmp_mean)
            ax.axvline(pouymayou_MLrisk[i,j]*100., 
                       color=usz_orange, label="Pouymayou et al");
            _, bins, _ = ax.hist(hmm_risk[i,j]*100., bins=bins, 
                                 histtype="stepfilled", density=True, 
                                 color=hmm_color) #, label="HMM sampling");
            ax.hist(bn_risk[i,j]*100., bins=bins, density=True, 
                    histtype="step", label="BN sampling", color=usz_blue);
            ax.tick_params(labelsize="xx-small")
            
            if i == 0:
                ax.set_title(f"{txt[j]}", 
                             fontsize="medium", fontweight="normal");
            else:
                ax.set_xlabel("risk [%]");
                
    else:
        ax = fig.add_subplot(spec[i,:])
        bins = np.linspace(0., 100., 150)
        ax.set_xlim(bins[0], bins[-1])
        ax.set_ylabel(f"{lvls[i]}");
        
        for j in range(4):
            tmp_mean = np.mean(hmm_risk[i,j])
            hmm_color = risk_map(tmp_mean)
            ax.axvline(pouymayou_MLrisk[i,j]*100., 
                       color=usz_orange, label="Pouymayou et al");
            n, bins, _ = ax.hist(hmm_risk[i,j]*100., bins=bins, 
                                 histtype="stepfilled", density=True, 
                                 color=hmm_color) #, label="HMM sampling");
            ax.hist(bn_risk[i,j]*100., bins=bins, density=True, 
                    histtype="step", label="BN sampling", color=usz_blue, 
                    linestyle='-');
            ax.set_xticks(np.linspace(0,100, 11))
            ax.tick_params(labelsize="xx-small")
            
            if ((i == 2) and (j == 0)):
                ax.text(x=100*pouymayou_MLrisk[i,j]-4*(bins[1]-bins[0]), 
                        y=0.35, 
                        s=txt[j])
            elif ((i == 1) and (j == 2)):
                ax.text(x=100*pouymayou_MLrisk[i,j]+2.5*(bins[1]-bins[0]), 
                        y=0.2, 
                        s=txt[j])
            else:
                ax.text(x=100*pouymayou_MLrisk[i,j]+(bins[1]-bins[0]), 
                        y=np.max(n)+0.05, 
                        s=txt[j])
                
if SAVE_FIGURES:
    plt.savefig("./figures/HMM_BN_risk_comparison.png", dpi=300, 
                bbox_inches="tight")
    plt.savefig("./figures/HMM_BN_risk_comparison.svg", 
                bbox_inches="tight")

The plot shows the good agreement between the different ways of computing the risk using different models and different inference techniques.

## Simultaneous Learning
If we learn both the system's parameters AND the center of the time prior at the same time. But the naive way just leads to overfitting and very unrealistic combinations of parameters. So, what we are doing here is fixing the time prior for the early stage learning (and use the sanguineti data with chosen N0-ratio) and learn the probability rates along with the time-prior for the late stage (where the data only consists of N0 / N+ patients).

In [ ]:
# abstract representation of the lymphatic network
graph = {'T'  : ['I', 'II', 'III', 'IV'], 
         'I'  : ['II'], 
         'II' : ['III'], 
         'III': ['IV'], 
         'IV' : []}

systm = lymph.System(graph=graph)

In [ ]:
# define specificity and sensitivity for diagnostic modalities
spsn_dict = {"path": [1., 1.]}

# generate FIRST PART of C matrix...
systm.load_data(data, t_stage=["early"], spsn_dict=spsn_dict, mode="HMM")

# ...and SECOND PART
systm.C_dict['late'] = np.zeros(shape=(len(systm.obs_list),2), dtype=int)
systm.f_dict['late'] = np.zeros(shape=(2,), dtype=int)

n_late = np.sum(systm.f_dict['early'])
# N0 of late stage
systm.C_dict['late'][0,0] = 1
systm.f_dict['late'][0] = int(n_late * (1 - (1. / 1.2)))  # 20% N0 for late T-stage
# N+ of late stage
systm.C_dict['late'][(np.sum(systm.obs_list, axis=1) > 0),1] = 1
systm.f_dict['late'][1] = int(n_late / 1.2)

In [ ]:
theta0 = np.random.uniform(size=(8,))
assert systm.combined_likelihood(theta0, t_stage=['early', 'late'])

In [ ]:
ndim, nwalker, nstep, burnin = 8, 200, 2000, 1000
np.random.seed(SEED)
theta0 = np.random.uniform(low=0., high=1., size=(nwalker,ndim))
moves = [(emcee.moves.DEMove(), 0.8), (emcee.moves.DESnookerMove(), 0.2)]

if DRAW_SAMPLES:
    if __name__ == "__main__":
        with Pool() as pool:
            sampler = emcee.EnsembleSampler(nwalker, ndim, systm.combined_likelihood, 
                                            args=[['early', 'late']], 
                                            moves=moves, pool=pool)
            sampler.run_mcmc(theta0, nstep, progress=True)

    # extracting 200,000 of the 400,000 samples
    samples_simultaneous = sampler.get_chain(flat=True, discard=burnin)

    # saving the sampled data to disk for later convenience
    np.save("./samples/simultaneous.npy", samples_simultaneous)
    
else:
    # loading in case we don't want to draw all the samples again
    samples_simultaneous = np.load("./samples/simultaneous.npy")

In [ ]:
if DRAW_SAMPLES:
    # check acceptance faction of the sampler to get an indication on whether sth
    # went wrong or not
    ar = np.mean(sampler.acceptance_fraction)
    print(f"the simultaneous sampler accepted {ar * 100 :.2f} % of samples.")

In [ ]:
labels = [r"$\tilde{b}_1$", r"$\tilde{b}_2$", 
          r"$\tilde{b}_3$", r"$\tilde{b}_4$", 
          r"$\tilde{t}_{12}$", r"$\tilde{t}_{23}$", 
          r"$\tilde{t}_{34}$", r"$p$"]

fig = plt.figure(figsize=set_size(width="full", ratio=1))
corner.corner(
    samples_simultaneous,
    labels=labels,
    smooth=True,
    fig=fig,
    color=usz_blue,
    hist_kwargs={'histtype': 'stepfilled', 'color': usz_blue},
    **{
        "plot_datapoints": False,
        "no_fill_contours": True,
        "plot_density": True,
        "contour_kwargs": {"colors": "black"},
        "levels": np.array([0.2, 0.5, 0.8])
    },
    show_titles=True,
    title_kwargs={"fontsize": "medium"}
);

if SAVE_FIGURES:
    plt.savefig("./figures/corner_simultaneous.png", dpi=300, 
                bbox_inches="tight")
    plt.savefig("./figures/corner_simultaneous.svg", 
                bbox_inches="tight")

In [ ]:
mean_late_p = np.mean(samples_simultaneous[:,7])

fig, ax = plt.subplots(1,2, figsize=set_size(width="full", ratio=1.61**2))

ax[0].axvline(0.4, color=usz_blue, linewidth=2, 
              label=r"$p_{\mathrm{early}}$ (fixed)");
ax[0].hist(samples_simultaneous[:,7], bins=40, density=True, 
           color=usz_red, histtype="stepfilled", 
           label=r"$p_{\mathrm{late}}$");
ax[0].set_xlabel("Binomial parameter $p$");
# ax[0].set_ylabel(r"$p\left(\theta_p^T\right)$", fontsize="large");
ax[0].legend();

t = np.arange(T_MAX+1)
ax[1].plot(t, sp.stats.binom.pmf(t, T_MAX, 0.4), 'o-', 
           label=r"$p_{\mathrm{early}}$ (fixed)");
ax[1].plot(t, sp.stats.binom.pmf(t, T_MAX, mean_late_p), 'o-', color=usz_red, 
           label=r"$\mathbb{E}[p_{\mathrm{late}}]$");

ax[1].legend();
ax[1].set_xlabel("Time step $t$");
ax[1].set_ylabel(r"$p_T(t)$");
ax[1].set_xlim([1,T_MAX]);

if SAVE_FIGURES:
    plt.savefig("./figures/simultaneous_learnedP.png", dpi=300, 
                bbox_inches="tight")
    plt.savefig("./figures/simultaneous_learnedP.svg", 
                bbox_inches="tight")

Now we can again compare risk predictions for different T-stages of disease.

In [ ]:
# sensitivity and specificity from [2]
graph = {'T'  : ['I', 'II', 'III', 'IV'], 
         'I'  : ['II'], 
         'II' : ['III'], 
         'III': ['IV'], 
         'IV' : []}

tst_systm = lymph.System(graph=graph)
tst_systm.set_modalities(spsn_dict={"CT": [0.76, 0.81]})

In [ ]:
np.random.seed(SEED)
nsubset = 20000
subset = samples_simultaneous[np.random.randint(low=0, 
                                                high=(nstep-burnin)*nwalker, 
                                                size=nsubset)]
risk_III = np.zeros(shape=(4, nsubset))
risk_IV = np.zeros(shape=(4, nsubset))

inv_III = np.array([None, None, 1, None])
obs_III = np.array([[0, 0, 0, 0],   # no involvement in lvl II observed
                    [0, 1, 0, 0]])  # involvement observed

inv_IV = np.array([None, None, None, 1])
obs_IV = np.array([[0, 0, 0, 0], 
                   [0, 1, 1, 0]])

for i, th in enumerate(subset):
    tst_systm.set_theta(th[:7])
    prior = np.vstack([sp.stats.binom.pmf(np.arange(T_MAX+1), T_MAX, 0.4), 
                       sp.stats.binom.pmf(np.arange(T_MAX+1), T_MAX, th[7])])
    
    for k in range(4):
        risk_III[k,i] = tst_systm.risk(inv=inv_III, 
                                       obs={"CT": obs_III[k % 2]}, 
                                       time_prior=prior[k // 2])
        risk_IV[k,i] = tst_systm.risk(inv=inv_IV, 
                                      obs={"CT": obs_IV[k % 2]}, 
                                      time_prior=prior[k // 2])

In [ ]:
fig, ax = plt.subplots(1,2, 
                       figsize=set_size(width="full", ratio=2*1.5))
kwargs = [{"histtype": "stepfilled", "alpha": 0.5}, 
          {"histtype": "step", "linewidth": 1.5}]
colors = [usz_blue, usz_orange]
time_label = ["early", "late"]
inv_III_label = ["no observed involvement", 
                 "only LNL II observed involved"]
inv_IV_label = ["no observed involvement", 
                "LNL II & III observed involved"]

for k in range(4):
    bins = np.linspace(0., 25., 50)
    ax[0].hist(100*risk_III[k], bins=bins, density=True, 
               color=colors[k // 2], **kwargs[k % 2], 
               label=f"{time_label[k // 2]} | {inv_III_label[k % 2]}")
    ax[0].set_xlabel("risk $R$ [%]");
    ax[0].set_ylabel(r"$p(R)$");
    ax[0].set_xlim([bins[0], bins[-1]])
    ax[0].legend()
    ax[0].set_title("III")
    
    bins = np.linspace(0., 15., 50)
    ax[1].hist(100*risk_IV[k], bins=bins, density=True, 
               color=colors[k // 2], **kwargs[k % 2], 
               label=f"{time_label[k // 2]} | {inv_IV_label[k % 2]}")
    ax[1].set_xlabel("risk $R$ [%]");
    ax[1].set_xlim([bins[0], bins[-1]]);
    ax[1].legend()
    ax[1].set_title("IV")
    
if SAVE_FIGURES:
    plt.savefig("./figures/simultaneous_risk.png", dpi=300, 
                bbox_inches="tight")
    plt.savefig("./figures/simultaneous_risk.svg", 
                bbox_inches="tight")

## References
1. <a class="anchor" id="sanguineti"></a>Sanguineti Giuseppe [et al.] Defining the risk of involvement for each neck nodal level in patients with early T-stage node-positive oropharyngeal carcinoma. [Journal] // International Journal of Radiation Oncology Biology Physics. - 2008. - 5 : Vol. 74. - pp. 1356-1364.
2. <a class="anchor" id="pouymayou"></a>Pouymayou Bertrand [et al.] A Bayesian network model of lymphatic tumor progression for personalized elective CTV definition in head and neck cancers [Journal] // Physics in Medicine and Biology. - 2019. - 16 : Vol. 64. - p. 165003.

## Appendix

### t-Dependence of Rates

Since the HMM-formalism has more parameters than the BN through its time prior, we expect the system to be somewhat overdetermined. In our case this means that we can basically choose an arbitrary number of time steps and the base and transition probability rates will essentially adapt to our choice. To see *how* the rates depend on the time prior's length we'll look at a simplistic example:

For the simplest example, the time prior $p_T(t,T)=1 / T$ is uniform and we're only looking at a system with one node that has empirically an involvement probability of $p^*$ (e.g. $0.4$), the base probability rate $p$ must become smaller, as the length of the uniform prior increases. More formally,

$$
\begin{align}
p^* 
&= \sum_{t=1}^{T}{\frac{1}{T}\begin{pmatrix} 1 & 0 \end{pmatrix}\begin{bmatrix} 1-p & p \\ 0 & 1 \end{bmatrix}^t\begin{pmatrix} 0 \\ 1 \end{pmatrix}} \\
&= \sum_{t=1}^{T}{\frac{1}{T}\begin{pmatrix} 1 & 0 \end{pmatrix}\begin{bmatrix} (1-p)^t & 1-(1-p)^t \\ 0 & 1 \end{bmatrix}\begin{pmatrix} 0 \\ 1 \end{pmatrix}} \\
&= \sum_{t=1}^{T}{\frac{1}{T}\sum_{k=1}^{t}{{t\choose k}p^k(1-p)^{t-k}}}
= \sum_{t=1}^{T}{\frac{1}{T}\left[ 1 - (1-p)^t \right]} \\
&= 1 - \frac{1}{T}\sum_{t=1}^{T}{(1-p)^t}
\end{align}
$$

Let's write $q = 1 - p$ and then the sum as

$$
s = \sum_{t=1}^{T}{q^t} = q + q^2 + \ldots + q^T
$$

hence

$$
q\cdot s = q^2 + q^3 + \ldots + q^{T+1}
$$

and

$$
\begin{align}
s - qs &= q - q^{T+1} \\
\Rightarrow\, s(1 - q) &= q(1 - q^T) \\
\Rightarrow\, s &= \frac{q(1 - q^T)}{1 - q}
\end{align}
$$

So we can write the empirical probability $p^*$ as

$$
p^* = 1 - \frac{q(1 - q^T)}{T(1 - q)} = 1 - \frac{(1-p)(1 - (1-p)^T)}{Tp}
$$

This cannot be solved analytically for $p$ in the case of arbitrary $T$, but it is easy to find numerical solutions.

In [ ]:
p_star = 0.4
max_prior_length = 15

Ts = np.linspace(1, max_prior_length, max_prior_length, dtype=int)
f = lambda p,t,p_star : 1 - (1 - p)*(1 - (1-p)**t) / (t*p) - p_star

sols = []
for t in Ts:
    # find roots of f numerically
    sols.append(sp.optimize.root_scalar(f, args=(t,p_star), bracket=[0.001, 0.999]).root)
    
# plot the computed solutions
fig, ax = plt.subplots(figsize=set_size())
ax.plot(Ts, sols, 'o--');
ax.set_xlim([0, max_prior_length+1]);
ax.set_xticks(Ts[::2]);
ax.set_xlabel("length of prior $T$");
ax.set_ylabel(r"solution for $\tilde{b}_1$");

if SAVE_FIGURES:
    plt.savefig("./figures/simple_rate_decay.png", dpi=300, 
                bbox_inches="tight")
    plt.savefig("./figures/simple_rate_decay.svg", 
                bbox_inches="tight")

To see if this overly simplified model applies to the full HMM as well, we trained our model for 15 different uniform time priors and plotted the learned mean parameters. We also compared the risk predictions using these different time priors.

In [ ]:
# sampling parameters
ndim, nwalker, nstep, burnin = 7, 200, 2000, 1000
moves = [(emcee.moves.DEMove(), 0.8), (emcee.moves.DESnookerMove(), 0.2)]

# starting point
np.random.seed(SEED)
theta0 = np.random.uniform(low=0., high=1., size=(nwalker,ndim))
prior_dict = {}

samples_multiT_HMM = np.zeros(shape=(max_prior_length, 
                                     nwalker*(nstep-burnin), 
                                     ndim))

for i in range(max_prior_length):
    if DRAW_SAMPLES:
        if __name__ == "__main__":
            with Pool() as pool:
                # uniform time prior of varying length
                prior_dict['early'] = np.concatenate([[0.], [1./(i+1)]*(i+1)])

                sampler = emcee.EnsembleSampler(nwalker, ndim, systm.likelihood, 
                                                args=[['early'], prior_dict], 
                                                kwargs={"mode": "HMM"},
                                                moves=moves, pool=pool)
                sampler.run_mcmc(theta0, nstep, progress=True)

                # store the i-th sampling round
                samples_multiT_HMM[i] = sampler.get_chain(flat=True, 
                                                          discard=burnin)

        # saving the sampled data to disk for later convenience
        np.save(f"./samples/multiT_HMM_{i}.npy", samples_multiT_HMM[i])
        
    else:
        # loading in case we don't want to draw all the samples again
        samples_multiT_HMM[i] = np.load(f"./samples/multiT_HMM_{i}.npy")

Plot the results vs the theoretical values. This is only applicable to the base probability rates $\tilde{b}_1$ and $\tilde{b}_2$, since all other LNL's probability rates are also influenced by other efferent spread pathways.

In [ ]:
# compute mean and variance of sampled parameters
multiT_b = [np.mean(samples_multiT_HMM[:,:,i], axis=1) for i in range(4)]
multiT_b_var = [np.var(samples_multiT_HMM[:,:,i], axis=1) for i in range(4)]
multiT_t = [np.mean(samples_multiT_HMM[1:,:,i], axis=1) for i in range(4,7)]
multiT_t_var = [np.var(samples_multiT_HMM[1:,:,i], axis=1) for i in range(4,7)]

# plot
fig, ax = plt.subplots(1,2, figsize=set_size("full", ratio=2.8))

# plot base prob sample means for differently lengthed time priors
for i in range(4):
    ax[0].plot(range(1,max_prior_length+1), multiT_b[i], "o", mfc="none", ms=4, 
               label=f"$\\tilde{{b}}_{{{i+1}}}$ sampled");

# prevalence of involvement in LNL I and II
prevalence_I = np.sum(data[("path", "I")].to_numpy()) / len(data)
prevalence_II = np.sum(data[("path", "II")].to_numpy()) / len(data)
theory_roots = np.zeros(shape=(2,max_prior_length))

# compute roots with P^* = prevalence
for t in range(1,max_prior_length+1):
    theory_roots[0,t-1] = sp.optimize.root_scalar(f, 
                                                  args=(t, prevalence_I), 
                                                  bracket=[0.001, 0.999]).root
    theory_roots[1,t-1] = sp.optimize.root_scalar(f, 
                                                  args=(t, prevalence_II), 
                                                  bracket=[0.001, 0.999]).root

# plot theoretical roots
ax[0].plot(range(1,max_prior_length+1), theory_roots[0], "-", alpha=0.5, 
           label=r"$\tilde{b}_{1}$ theory");
ax[0].plot(range(1,max_prior_length+1), theory_roots[1], "-", alpha=0.5, 
           label=r"$\tilde{b}_{2}$ theory");

offset = 0.5
ax[0].set_xlim([1 - offset, max_prior_length + offset]);
ax[0].set_xticks(np.arange(1, max_prior_length+1, 2))
ax[0].set_xlabel("length of prior $T$");
ax[0].set_ylabel(r"Base Probability Rate $\tilde{b}$");
ax[0].legend(ncol=1);


# plot trans prob sample means
# plot sample means for differently lengthed time priors
for i in range(3):
    ax[1].plot(range(2,max_prior_length+1), multiT_t[i], "o", mfc="none", ms=4, 
               label=f"$\\tilde{{t}}_{{{i+1}}}$ sampled");
    
offset = 0.5
ax[1].set_xlim([1 - offset, max_prior_length + offset]);
ax[1].set_xticks(np.arange(1, max_prior_length+1, 2))
ax[1].set_xlabel("length of prior $T$");
ax[1].set_ylabel(r"Transition Probability Rate $\tilde{t}$");
ax[1].legend(ncol=1);

if SAVE_FIGURES:
    plt.savefig("./figures/rate_decay_theory_vs_sampled.png", dpi=300, 
                bbox_inches="tight")
    plt.savefig("./figures/rate_decay_theory_vs_sampled.svg", 
                bbox_inches="tight")

These plots show that the probability rates of a more complex and realistic system qualitatively follow the same behaviour as the example with only one node. This serves as an argument why we can essentially choose the length of the prior as it suits us.

Finally, let's check if the risk actually changes:

In [ ]:
graph = {'T'  : ['I', 'II', 'III', 'IV'], 
         'I'  : ['II'], 
         'II' : ['III'], 
         'III': ['IV'], 
         'IV' : []}

systm = lymph.System(graph=graph)

# set specificity & sensitivity of diagnostic modality (here CT) manually
spsn_dict = {"CT": [0.76, 0.81]}
systm.set_modalities(spsn_dict=spsn_dict)

In [ ]:
# what do we want to know, what do we know?
inv = np.array([None, None, 1, None])  # we're interested in the risk of lvl 3 being involved
# our observation is that lvl 2 is involved
obs = {"CT": np.array([0, 1, 0, 0])}

np.random.seed(SEED)
ndim, nwalker, nstep, burnin = 7, 200, 2000, 1000
thin = 200
time_prior_dict = {}
hmm_risk = np.zeros(shape=(max_prior_length, (nstep-burnin)*nwalker//thin))

for k in range(max_prior_length):
    # time priors
    time_prior_dict['early'] = np.concatenate([[0.], [1./(k+1)] * (k+1)])

    # risk for HMM and two different "T-stages" (early and late)
    subset = np.random.permutation(samples_multiT_HMM[k])[::thin]
    for i, sample in enumerate(subset):
        systm.set_theta(sample)
        hmm_risk[k, i] = systm.risk(inv=inv, obs=obs, 
                                    time_prior=time_prior_dict["early"], 
                                    mode="HMM")

In [ ]:
bn_risk = np.zeros(shape=(len(samples_BN) // thin))
for i, sample in enumerate(np.random.permutation(samples_BN)[::thin]):
    systm.set_theta(sample)
    bn_risk[i] = systm.risk(inv=inv, obs=obs, mode="BN")

In [ ]:
fig, ax = plt.subplots(figsize=set_size())

ax.hist(100*hmm_risk[0], bins=100, range=[0., 50], density=True, 
        color=usz_red, alpha=0.1, label=f"HMM ({max_prior_length} different $T$)")
[ax.hist(100*hmm_risk[k], bins=100, range=[0., 50], density=True, 
         color=usz_red, alpha=0.1) for k in np.arange(1,max_prior_length)];
ax.hist(100*bn_risk, histtype="step", bins=100, linewidth=2, range=[0., 50], 
        density=True, label="BN");
ax.set_xlim([0., 20]);
ax.legend();
ax.set_xlabel(r"risk $R$ [%]");
ax.set_ylabel("PDF of risk");

if SAVE_FIGURES:
    plt.savefig("./figures/multi_length_risk.png", dpi=300, 
                bbox_inches="tight")
    plt.savefig("./figures/multi_length_risk.svg", 
                bbox_inches="tight")

The risk for all the HMMs is the same, except for the one that only includes one time step. The reason for this is that when the system is not given the time to spread, it cannot correctly estimate the conditional risks of what would happen if the previous node was involved vs if it was not involved. So the HMM that is "too short" overestimates the risk due to the high prevalence of LNL III's involvement.